# Факторизация

- Факторизация
- Специальные матричные структуры
- Общая линейная алгебра

Прежде чем мы начнем, давайте настроим линейную систему и используем `LinearAlgebra`, чтобы ввести факторизации и специальные матричные структуры.

In [ ]:
import Pkg
Pkg.add("LinearAlgebra")

In [ ]:
using LinearAlgebra

In [ ]:
A = rand(3, 3)

In [ ]:
x = fill(1, (3,))

In [ ]:
b = A * x

## Факторизация

#### [LUР-факторизация](https://ru.wikipedia.org/wiki/LUP-%D1%80%D0%B0%D0%B7%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5)

```julia
PA = LU
``` 
где `P` матрица перестановок, `L` нижняя треугольная матрица и `U` - верхняя.

LU-разложение используется для решения *систем линейных уравнений*, *обращения матриц* и *вычисления определителя*.
По сравнению с алгоритмом LU-разложения алгоритм LUP-разложения может обрабатывать любые невырожденные матрицы и при этом обладает более высокой вычислительной устойчивостью.

Julia позволяет вычислять  LU-факторизацию и определяет составной тип факторизации для его хранения.

In [ ]:
Alu = lu(A)

In [ ]:
typeof(Alu)

Различные части факторизации могут быть извлечены путем доступа к их специальным свойствам

In [ ]:
Alu.P

In [ ]:
Alu.p

In [ ]:
Alu.L

In [ ]:
Alu.U

Julia может отправлять методы на объекты факторизации. 

Например, мы можем решить линейную систему, используя либо исходную матрицу, либо объект факторизации.

In [ ]:
A\b

In [ ]:
Alu\b

Точно так же мы можем вычислить определитель `A`, используя либо` A`, либо объект факторизации

In [ ]:
det(A)

In [ ]:
det(Alu)

#### [QR-факторизация](https://ru.wikipedia.org/wiki/QR-%D1%80%D0%B0%D0%B7%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5)

```
A=QR
``` 

где `Q` унитарная/ортогональная матрица и `R` верхнетреугольная матрица.

QR-разложение используется для поиска *собственных векторов и чисел* матрицы.

In [ ]:
Aqr = qr(A)

Подобно факторизации LU, матрицы `Q` и` R` могут быть извлечены из объекта факторизации QR через

In [ ]:
Aqr.Q

In [ ]:
Aqr.R

Покажем, что $Q$ - ортогональная матрица.

In [ ]:
Aqr.Q' * Aqr.Q

#### [Спектральное разложение матрицы](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B5%D0%BA%D1%82%D1%80%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D1%80%D0%B0%D0%B7%D0%BB%D0%BE%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5_%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%86%D1%8B)

Представление квадратной матрицы $A$ в виде произведения трёх матриц: $A=V\Lambda V^{-1}$, где $V$ - матрица, столбцы которой являются собственными векторами матрицы $A$, $\Lambda$ - диагональная матрица с соответствующими собственными значениями на главной диагонали, $V^{-1}$ - матрица, обратная матрице $V$.

Результаты собственных разложений, разложения по сингулярным числам, разложения по Гессенбергу и разложения Шура хранятся в типах `Factorization`. 

Собственная декомпозиция может быть вычислена

In [ ]:
Asym = A + A'

In [ ]:
AsymEig = eigen(Asym)

Значения и векторы могут быть извлечены из типа Eigen с помощью специальной индексации

In [ ]:
AsymEig.values

In [ ]:
AsymEig.vectors

Еще раз, когда факторизация хранится в типе, мы можем отправить его и написать специальные методы, которые используют свойства факторизации, например, $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [ ]:
inv(AsymEig)*Asym

## Специальные матричные структуры

Матричная структура очень важна в линейной алгебре. Чтобы понять, насколько это важно, давайте работать с большой линейной системой.

In [ ]:
n = 1000
A = randn(n,n)

Средства Julia можно узнавать принадлежность объекта к тем или иным специальным матричным структурам

In [ ]:
Asym = A + A'

In [ ]:
issymmetric(Asym)

но иногда ошибка с плавающей точкой может помешать.

In [ ]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

In [ ]:
issymmetric(Asym_noisy)

К счастью, мы можем объявить структуру явно с помощью, например, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` и `SymTridiagonal`.

In [ ]:
Asym_explicit = Symmetric(Asym_noisy)

Давайте сравним, как долго Юлия вычисляет собственные значения `Asym`, `Asym_noisy`, и `Asym_explicit`

In [ ]:
import Pkg
Pkg.add("BenchmarkTools")

In [ ]:
using BenchmarkTools

In [ ]:
@btime eigvals(Asym);

In [ ]:
@btime eigvals(Asym_noisy);

In [ ]:
@btime eigvals(Asym_explicit);

В этом примере, используя `Symmetric()` на `Asym_noisy` мы получаем `5x`-кратный прирост эффективности.

#### Большая проблема

Использование типов `Tridiagonal` и` SymTridiagonal` для хранения трехдиагональных матриц позволяет работать с потенциально очень большими трехдиагональными задачами.

In [ ]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1))

In [ ]:
@btime eigmax(A)

Эта задача не может быть решена, если матрица должна храниться в виде (плотного) типа `Matrix`.

In [ ]:
B = Matrix(A)

## Общая линейная алгебра

Обычный способ добавить поддержку числовой линейной алгебры - это обернуть подпрограммы *BLAS* и *LAPACK*. Собственно, для матриц с элементами `Float32`,` Float64`, `Complex {Float32}` или `Complex {Float64}` разработчики Julia использовали такое же решение. Однако Julia также поддерживает общую линейную алгебру, что позволяет, например, работать с матрицами и векторами рациональных чисел.

#### Рациональные числа

Julia имеет встроенные рациональные числа. Чтобы задать рациональное число, используйте двойные косые черты:

In [ ]:
1//2

#### Пример: рациональная линейная система в уравнениях

В следующем примере показано, как можно решить линейную систему уравнений с рациональными элементами без преобразования в типы элементов с плавающей запятой. Переполнение может легко стать проблемой при работе с рациональными числами, поэтому мы используем BigInt.

In [ ]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

In [ ]:
x = fill(1, 3)
b = Arational*x

In [ ]:
Arational\b

In [ ]:
lu(Arational)

### Упражнения

---
#### 1
Каковы собственные значения матрицы A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
присвойте их переменной `A_eigv`

In [ ]:
using LinearAlgebra

In [ ]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

---
#### 2 
Создать диагональную матрицу из собственных значений `A`.

In [ ]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]

---
#### 3 
Создайте `LowerTriangular` матрицу из `A` и запишите её в `A_lowertri`

In [ ]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]